# Project task 02: Restaurant recommendation

In [1]:
import scipy.sparse as sp
import numpy as np
import scipy
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression, Ridge
import time

The goal of this task is to recommend restaurants to users based on the rating data in the Yelp dataset. For this, we try to predict the rating a user will give to a restaurant they have not been to yet based on a latent factor model.

## 1. Load data

First download `ratings.npy` from Piazza ([download link](https://syncandshare.lrz.de/dl/fiKMoxRNusLoFpFHkXXEgvdZ/ratings.npy)).

In [2]:
ratings = np.load("ratings.npy")

In [3]:
# We have triplets of (user, restaurant, rating).
ratings

array([[101968,   1880,      1],
       [101968,    284,      5],
       [101968,   1378,      2],
       ...,
       [ 72452,   2100,      4],
       [ 72452,   2050,      5],
       [ 74861,   3979,      5]], dtype=uint32)

Now we transform the data into a matrix of dimension [N, D], where N is the number of users and D is the number of restaurants in the dataset.  
We **strongly recommend** to load the data as a sparse matrix to avoid out-of-memory issues.

In [4]:
# Store the matrix into the variable M

### YOUR CODE HERE ###
ratings_matrix = sp.csr_matrix((ratings[:,2], (ratings[:,0], ratings[:,1])))
ratings_matrix

<337867x5899 sparse matrix of type '<class 'numpy.uint32'>'
	with 929606 stored elements in Compressed Sparse Row format>

## 2. Preprocess the data

In the preprocessing step, we recursively remove all users and restaurants with 10 or less ratings. 

Then, we randomly select 200 data points for the validation and test sets, respectively.

After this, we subtract the mean rating for each users to account for this global effect.   
**Hint**: Some entries might become zero in this process -- but these entries are different than the 'unknown' zeros in the matrix. Store the indices of which we have data available in a separate variable.

In [5]:
#function to remove rows from a sparse matrix given indices
def delete_rows_csr1(mat, indices):
    """
    Remove the rows denoted by ``indices`` form the CSR sparse matrix ``mat``.
    """
    if not isinstance(mat, sp.csr_matrix):
        raise ValueError("works only for CSR format -- use .tocsr() first")
    indices = list(indices)
    mask = np.ones(mat.shape[0], dtype=bool)
    mask[indices] = False
    return mat[mask]

def delete_cols_csr1(mat, indices):
    """
    Remove the cols denoted by ``indices`` form the CSR sparse matrix ``mat``.
    """
    if not isinstance(mat, sp.csr_matrix):
        raise ValueError("works only for CSR format -- use .tocsr() first")
    indices = list(indices)
    mask = np.ones(mat.shape[1], dtype=bool)
    mask[indices] = False
    return mat[:,mask]

In [6]:
def cold_start_preprocessing(matrix, min_entries):
    """
    Recursively removes rows and columns from the input matrix which have less than min_entries nonzero entries.
    
    Parameters
    ----------
    matrix      : sp.spmatrix, shape [N, D]
                  The input matrix to be preprocessed.
    min_entries : int
                  Minimum number of nonzero elements per row and column.

    Returns
    -------
    matrix      : sp.spmatrix, shape [N', D']
                  The pre-processed matrix, where N' <= N and D' <= D
        
    """
    print("Shape before: {}".format(matrix.shape))
    
    #matrix = np.asmatrix(matrix.toarray())
#     while(~((((matrix>0).sum(0).A1 > min_entries).all())&(((matrix>0).sum(1).A1 > min_entries).all()))):
#     rowise = np.delete(matrix, np.where(((matrix>0).sum(1))<=min_entries), 0)
#     colwise = np.delete(rowise, np.where(((rowise>0).sum(0))<=min_entries), 1)
#     matrix = colwise
    #If above while loop doesnt work please run the below commented while loop
    while(~((((matrix>0).sum(0) > min_entries).all())&(((matrix>0).sum(1) > min_entries).all()))):
        rr =  delete_rows_csr1(matrix,np.where((matrix>0).sum(1) <=min_entries)[0] )
        cc1 = delete_cols_csr1(rr,np.where((rr>0).sum(0) <=min_entries)[1])
        matrix = cc1
        
    #matrix = sp.csr_matrix(matrix)
    new_shape = matrix.shape
    
    print("Shape after: {}".format(matrix.shape))
    
    nnz = matrix>0
    
    assert (nnz.sum(0).A1 > min_entries).all()
    assert (nnz.sum(1).A1 > min_entries).all()
    return matrix, new_shape

In [7]:
def shift_user_mean(matrix):
    """
    Subtract the mean rating per user from the non-zero elements in the input matrix.
    
    Parameters
    ----------
    matrix : sp.spmatrix, shape [N, D]
             Input sparse matrix.
    Returns
    -------
    matrix : sp.spmatrix, shape [N, D]
             The modified input matrix.
    
    user_means : np.array, shape [N, 1]
                 The mean rating per user that can be used to recover the absolute ratings from the mean-shifted ones.

    """
    
    ### YOUR CODE HERE ###
    reduced_matrix = matrix
    #if np.diff(reduced_matrix.indptr)>0:
    user_means = np.array(reduced_matrix.sum(axis=1).squeeze())[0]/np.diff(reduced_matrix.indptr)
    user_means[ ~ np.isfinite( user_means )] = 0 
    #print(type(user_means))
    #print(user_means)
    #else:
    #user_means = 0
    #matrix_mean_diag = sp.diags(user_means,0)
    matrix_mean_diag = np.diag(user_means)
    #print(matrix_mean_diag.shape)
    reduced_matrix_copy = reduced_matrix.copy()
    reduced_matrix_copy.data = np.ones_like(reduced_matrix_copy.data)
    matrix_mean_diag = sp.csr_matrix(matrix_mean_diag)
    M_demean = (reduced_matrix-(matrix_mean_diag*reduced_matrix_copy).todense())
    assert np.all(np.isclose(M_demean.mean(1), 0))
    return M_demean, user_means
    #assert np.all(np.isclose(matrix.mean(1), 0))


In [8]:
def split_data(matrix, n_validation, n_test):
    """
    Extract validation and test entries from the input matrix. 
    
    Parameters
    ----------
    matrix          : sp.spmatrix, shape [N, D]
                      The input data matrix.
    n_validation    : int
                      The number of validation entries to extract.
    n_test          : int
                      The number of test entries to extract.

    Returns
    -------
    matrix_split    : sp.spmatrix, shape [N, D]
                      A copy of the input matrix in which the validation and test entries have been set to zero.
    
    val_idx         : tuple, shape [2, n_validation]
                      The indices of the validation entries.
    
    test_idx        : tuple, shape [2, n_test]
                      The indices of the test entries.
    
    val_values      : np.array, shape [n_validation, ]
                      The values of the input matrix at the validation indices.
                      
    test_values     : np.array, shape [n_test, ]
                      The values of the input matrix at the test indices.

    """
    
    ### YOUR CODE HERE ###
    #matrix = matrix[0]
    nontrain_ind = np.random.choice(matrix.shape[0], size=(n_validation+n_test), replace=False)

    nontrain_mat = matrix[nontrain_ind, :]
    val_rows = np.random.choice(nontrain_ind.shape[0], size=n_validation, replace=False)
    val_mat = nontrain_mat[val_rows, :]
    val_ind = nontrain_ind[val_rows]
    
    test_mat = delete_rows_csr1(nontrain_mat.tocsr(), val_rows)
    test_ind = np.delete(nontrain_ind,val_rows,0)
    test_idx = []
#     t = test_mat.nonzero()
#     test_idx = list(zip(t[0],t[1]))
    test_values = []
    
    for i in range(len(test_ind)):
        for j in range(matrix.shape[1]):
            #print(test_ind[i])
            test_idx.append((test_ind[i], j))

    test_idx = tuple(test_idx)
    
    for i in range(len(test_idx)):
        test_values.append(matrix[test_idx[i]])
    test_values = np.array(test_values)

    val_idx = []
    for i in range(len(val_ind)):
        for j in range(matrix.shape[1]):
            val_idx.append((val_ind[i], j))

    val_idx = tuple(val_idx)
#     v = val_mat.nonzero()
#     val_idx = list(zip(v[0],v[1]))
    val_values = []
    for i in range(len(val_idx)):
        val_values.append(matrix[val_idx[i]])
    val_values = np.array(val_values)

    matrix[nontrain_ind, :] = 0
    
    matrix_split = sp.csr_matrix(matrix)

    matrix_split.eliminate_zeros()
    return matrix_split, val_idx, test_idx, val_values, test_values

In [9]:
M = ratings_matrix
M = cold_start_preprocessing(M, 10)

Shape before: (337867, 5899)
Shape after: (11275, 3531)


In [10]:
from copy import deepcopy
n_validation = 200
n_test = 200
# Split data
M1 = deepcopy(M[0])
M_train, val_idx, test_idx, val_values, test_values = split_data(M[0], n_validation, n_test)

/home/dp/miniconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [11]:
validation_matrix = np.array([M1[lst] for lst in val_idx])
validation_matrix = np.reshape(validation_matrix,(n_validation,M1.shape[1]))
test_matrix = np.array([M1[lst] for lst in test_idx])
test_matrix = np.reshape(test_matrix,(n_test,M1.shape[1]))
#print(validation_matrix.shape,test_matrix.shape)

In [12]:
# Store away the nonzero indices of M before subtracting the row means.
v = M[0].nonzero()
nonzero_indices =  list(zip(v[0],v[1]))
# Remove user means.
M_train = sp.csr_matrix(M_train)
M_shifted, train_user_means = shift_user_mean(M_train)
# Apply the same shift to the validation and test data.
validation_matrix = sp.csr_matrix(validation_matrix)
test_matrix = sp.csr_matrix(test_matrix)
val_values_shifted,val_user_means = shift_user_mean(validation_matrix)
test_values_shifted,test_user_means = shift_user_mean(test_matrix)

/home/dp/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


## 3. Alternating optimization

In the first step, we will approach the problem via alternating optimization, as learned in the lecture.

In [13]:
def initialize_Q_P(matrix, k, init='random'):
    """
    Initialize the matrices Q and P for a latent factor model.
    
    Parameters
    ----------
    matrix : sp.spmatrix, shape [N, D]
             The matrix to be factorized.
    k      : int
             The number of latent dimensions.
    init   : str in ['svd', 'random'], default: 'random'
             The initialization strategy. 'svd' means that we use SVD to initialize P and Q, 'random' means we initialize
             the entries in P and Q randomly in the interval [0, 1).

    Returns
    -------
    Q : np.array, shape [N, k]
        The initialized matrix Q of a latent factor model.

    P : np.array, shape [k, D]
        The initialized matrix P of a latent factor model.
    """

    
    if init == 'svd':
        U, s, Vh = np.linalg.svd(matrix, full_matrices=False)
        Q = np.dot(U,np.diag(s))
    elif init == 'random':
    ### YOUR CODE HERE ###
        Q = np.random.normal(size=(matrix.shape[0], k))
        P = np.random.normal(size=(k, matrix.shape[1]))
    else:
        raise ValueError
        
    assert Q.shape == (matrix.shape[0], k)
    assert P.shape == (k, matrix.shape[1])
    return Q, P

In [49]:
def latent_factor_alternating_optimization(M, non_zero_idx, k, val_idx, val_values,
                                           reg_lambda, max_steps=100, init='random',
                                           log_every=1, patience=10, eval_every=1):
    """
    Perform matrix factorization using alternating optimization. Training is done via patience,
    i.e. we stop training after we observe no improvement on the validation loss for a certain
    amount of training steps. We then return the best values for Q and P oberved during training.
    
    Parameters
    ----------
    M                 : sp.spmatrix, shape [N, D]
                        The input matrix to be factorized.
                      
    non_zero_idx      : np.array, shape [nnz, 2]
                        The indices of the non-zero entries of the un-shifted matrix to be factorized. 
                        nnz refers to the number of non-zero entries. Note that this may be different
                        from the number of non-zero entries in the input matrix M, e.g. in the case
                        that all ratings by a user have the same value.
    
    k                 : int
                        The latent factor dimension.
    
    val_idx           : tuple, shape [2, n_validation]
                        Tuple of the validation set indices.
                        n_validation refers to the size of the validation set.
                      
    val_values        : np.array, shape [n_validation, ]
                        The values in the validation set.
                      
    reg_lambda        : float
                        The regularization strength.
                      
    max_steps         : int, optional, default: 100
                        Maximum number of training steps. Note that we will stop early if we observe
                        no improvement on the validation error for a specified number of steps
                        (see "patience" for details).
                      
    init              : str in ['random', 'svd'], default 'random'
                        The initialization strategy for P and Q. See function initialize_Q_P for details.
    
    log_every         : int, optional, default: 1
                        Log the training status every X iterations.
                    
    patience          : int, optional, default: 10
                        Stop training after we observe no improvement of the validation loss for X evaluation
                        iterations (see eval_every for details). After we stop training, we restore the best 
                        observed values for Q and P (based on the validation loss) and return them.
                      
    eval_every        : int, optional, default: 1
                        Evaluate the training and validation loss every X steps. If we observe no improvement
                        of the validation error, we decrease our patience by 1, else we reset it to *patience*.

    Returns
    -------
    best_Q            : np.array, shape [N, k]
                        Best value for Q (based on validation loss) observed during training
                      
    best_P            : np.array, shape [k, D]
                        Best value for P (based on validation loss) observed during training
                      
    validation_losses : list of floats
                        Validation loss for every evaluation iteration, can be used for plotting the validation
                        loss over time.
                        
    train_losses      : list of floats
                        Training loss for every evaluation iteration, can be used for plotting the training
                        loss over time.                     
    
    converged_after   : int
                        it - patience*eval_every, where it is the iteration in which patience hits 0,
                        or -1 if we hit max_steps before converging. 

    """

    ### YOUR CODE HERE ###
    loss = []
    
    for iter in range(10):
        loss.append(((M[M>0]-Q.dot(P)[M>0])**2).sum())
        if verbose:
            print('loss', loss[-1])
        # fix Q and update P
        for movie_idx in range(M.shape[1]):
            nnz_idx = np.argwhere(M[:, movie_idx]).flatten()
            res =np.linalg.inv(Q[nnz_idx].T.dot(Q[nnz_idx])).dot(Q[nnz_idx].T.dot(M[nnz_idx, movie_idx]))
            P[:, movie_idx] = res

        # fix  P and update Q
        for user_idx in range(M.shape[0]):
            nnz_idx = np.argwhere(M[user_idx, :]).flatten()
            res =np.linalg.inv(P[:, nnz_idx].dot(P[:, nnz_idx].T)).dot(P[:, nnz_idx].dot(M[user_idx, nnz_idx]))
            Q[user_idx, :] = res
    return loss
    
#     Q,P = initialize_Q_P(M, k, init='random')
#     print(Q.shape,P.shape)
#     v = M.nonzero()
#     nonzero_train_indices =  list(zip(v[0],v[1]))
#     M_train_binary = np.zeros((M.shape[0],M.shape[1]))
#     for lst in nonzero_train_indices:
#         M_train_binary[lst] = 1
        
#     def get_error(M, Q, P, M_train_binary):
#         # This calculates the MSE of nonzero elements
#         return np.sum((M_train_binary * np.asarray(M - np.dot(Q, P))) ** 2) / np.sum(M_train_binary)
    
    
#     print ("Starting Iterations")
    
#     MSE_list = []
    
#     for iter in range(max_steps):
#         for i, Mi in enumerate(M_train_binary):
#             print (i)
            
#             temp = np.dot(P, np.dot(np.diag(Mi), P.T)) + reg_lambda * np.eye(k)
#             #print(temp.shape)
#             TEMP2 = (np.diag(Mi)).shape
#             TEMP3 = M[i].T.shape
#             #print(TEMP2,TEMP3)
#             temp1 = np.dot(P, np.dot((np.diag(Mi)), M[i].T))
#             #print(temp1.shape)
#             Q[i] = np.linalg.solve(np.dot(P, np.dot(np.diag(Mi), P.T)) + reg_lambda * np.eye(k),
#                                        np.dot(P,np.dot(np.asmatrix(np.diag(Mi)), M[i].T))).T
#             print ("Error after solving for Q:", get_error(M, Q, P, M_train_binary))

#         for j, Mj in enumerate(M_train_binary.T):
#             print(j)
#             print((np.diag(Mj)).shape)
#             P[:,j] = list(np.linalg.solve(np.dot(Q.T, np.dot(np.diag(Mj), Q)) + reg_lambda * np.eye(k),
#                                      np.dot(Q.T, np.dot(np.diag(Mj), M[:, j]))))
#             print ("Error after solving for P:", get_error(M, Q, P, M_train_binary))

#         MSE_List.append(get_error(M, Q, P, M_train_binary))
#         print ('%sth iteration is complete...' % iter)

#     print (MSE_List)
    
    
    
    #return best_Q, best_P, validation_losses, train_losses, converged_after


#### Train the latent factor model with alternating optimization.

a) Learn the optimal $P$ and $Q$ using alternating optimization. That is, during each iteration you first update $Q$ while having $P$ fixed and then vice versa. Run the alternating optimization algorithm with $k=100$ and $\lambda=1$. Plot the training and validation losses over time.

In [54]:
latent_factor_alternating_optimization(M_shifted, nonzero_indices, 
                                                                           k=100, val_idx=val_idx,
                                                                           val_values=val_values_shifted, 
                                                                           reg_lambda=1, init='random',
                                                                           max_steps=100, patience=10)

(11275, 100) (100, 3531)
Starting Iterations
0
Error after solving for User Matrix: 1.20150091507
1
Error after solving for User Matrix: 1.20149978438
2
Error after solving for User Matrix: 1.201498618
3
Error after solving for User Matrix: 1.201498618
4
Error after solving for User Matrix: 1.20149811623
5
Error after solving for User Matrix: 1.20149771081
6


KeyboardInterrupt: 

#### Plot the validation and training losses over (training) time

In [ ]:

### YOUR PLOTTING CODE HERE ###
np.linalg.solve(np.dot(P, np.dot(np.diag(Mi), P.T)) + reg_lambda * np.eye(k),
                                       np.dot(P, np.dot(np.asmatrix(np.diag(M_train_binary)).T, M[i]))).T

b) (**Optional**): Try some different latent dimensions $k$ in the range [5, 100]. What do you observe (convergence time, final training/validation losses)?

In [ ]:
Q_a_2, P_a_2, val_l_a_2, tr_l_a_2, conv_a_2 = latent_factor_alternating_optimization(M_shifted, nonzero_indices, 
                                                                           k=20, val_idx=val_idx,
                                                                           val_values=val_values_shifted, 
                                                                           reg_lambda=0.1, init='random',
                                                                           max_steps=100, patience=10)

#### Plot the validation and training losses over (training) time

In [ ]:

### YOUR PLOTTING CODE HERE ###


## 4. Latent factorization using gradient descent

We now use gradient descent to factorize our ratings matrix. We will try both (mini-) batch and stochastic gradient descent. You can use the following equations for your implementation.

Recall that the objective function (loss) we wanted to optimize was:
$$
\mathcal{L} = \min_{P, Q} \sum_{(x, i) \in W} (r_{xi} - \mathbf{q}_i^T\mathbf{p}_x)^2 + \lambda_1\sum_x{\left\lVert \mathbf{p}_x  \right\rVert}^2 + \lambda_2\sum_i {\left\lVert\mathbf{q}_i  \right\rVert}^2
$$

where $W$ is the set of $(x, i)$ pairs for which $r_{xi}$ is known (in this case our known play counts). Here we have also introduced two regularization terms to help us with overfitting where $\lambda_1$ and $\lambda_2$ are hyper-parameters that control the strength of the regularization.

Naturally optimizing with gradient descent involves computing the gradient of the loss function $\mathcal{L}$ w.r.t. to the parameters. To help you solve the task we provide the following:

$$
\frac{\partial ((r_{xi} - \mathbf{q}_i^T\mathbf{p}_x)^2)}{\partial \mathbf{p}_x} = -2(r_{xi} - \mathbf{q}_i^T\mathbf{p}_x)\mathbf{q}_i\;, ~~~
\frac{\partial ((r_{xi} - \mathbf{q}_i^T\mathbf{p}_x)^2)}{\partial \mathbf{q}_i} = -2(r_{xi} - \mathbf{q}_i^T\mathbf{p}_x)\mathbf{p}_x 
$$

$$
\frac{\partial(\lambda_1{\left\lVert \mathbf{p}_x \right\rVert}^2)}{\partial \mathbf{p}_x} = 2 \lambda_1 \mathbf{p_x} \;, ~~~
\frac{\partial(\lambda_2{\left\lVert \mathbf{q}_i \right\rVert}^2)}{\partial \mathbf{q}_i} = 2 \lambda_2 \mathbf{q_i}
$$

**Hint**: You have to carefully consider how to combine the given partial gradients depending
on which variants of gradient descent you are using.  
**Hint 2**: It may be useful to scale the updates to $P$ and $Q$ by $\frac{1}{batch\_size}$ (in the case of full-sweep updates, this would be $\frac{1}{n\_users}$ for $Q$ and $\frac{1}{n\_restaurants}$ for $P$).


For each of the gradients descent variants you try report and compare the following:
* How many iterations do you need for convergence.
* Plot the loss (y axis) for each iteration (x axis).


In [51]:
def latent_factor_gradient_descent(M, non_zero_idx, k, val_idx, val_values, 
                                   reg_lambda, learning_rate, batch_size=-1,
                                   max_steps=50000, init='random',
                                   log_every=1000, patience=20,
                                   eval_every=50):
    """
    Perform matrix factorization using gradient descent. Training is done via patience,
    i.e. we stop training after we observe no improvement on the validation loss for a certain
    amount of training steps. We then return the best values for Q and P oberved during training.
    
    Parameters
    ----------
    M                 : sp.spmatrix, shape [N, D]
                        The input matrix to be factorized.
                      
    non_zero_idx      : np.array, shape [nnz, 2]
                        The indices of the non-zero entries of the un-shifted matrix to be factorized. 
                        nnz refers to the number of non-zero entries. Note that this may be different
                        from the number of non-zero entries in the input matrix M, e.g. in the case
                        that all ratings by a user have the same value.
    
    k                 : int
                        The latent factor dimension.
    
    val_idx           : tuple, shape [2, n_validation]
                        Tuple of the validation set indices.
                        n_validation refers to the size of the validation set.
                      
    val_values        : np.array, shape [n_validation, ]
                        The values in the validation set.
                      
    reg_lambda        : float
                        The regularization strength.

    learning_rate     : float
                        Step size of the gradient descent updates.
                        
    batch_size        : int, optional, default: -1
                        (Mini-) batch size. -1 means we perform standard full-sweep gradient descent.
                        If the batch size is >0, use mini batches of this given size.
                        
    max_steps         : int, optional, default: 100
                        Maximum number of training steps. Note that we will stop early if we observe
                        no improvement on the validation error for a specified number of steps
                        (see "patience" for details).
                      
    init              : str in ['random', 'svd'], default 'random'
                        The initialization strategy for P and Q. See function initialize_Q_P for details.
    
    log_every         : int, optional, default: 1
                        Log the training status every X iterations.
                    
    patience          : int, optional, default: 10
                        Stop training after we observe no improvement of the validation loss for X evaluation
                        iterations (see eval_every for details). After we stop training, we restore the best 
                        observed values for Q and P (based on the validation loss) and return them.
                      
    eval_every        : int, optional, default: 1
                        Evaluate the training and validation loss every X steps. If we observe no improvement
                        of the validation error, we decrease our patience by 1, else we reset it to *patience*.
                        
    Returns
    -------
    best_Q            : np.array, shape [N, k]
                        Best value for Q (based on validation loss) observed during training
                      
    best_P            : np.array, shape [k, D]
                        Best value for P (based on validation loss) observed during training
                      
    validation_losses : list of floats
                        Validation loss for every evaluation iteration, can be used for plotting the validation
                        loss over time.
                        
    train_losses      : list of floats
                        Training loss for every evaluation iteration, can be used for plotting the training
                        loss over time.                     
    
    converged_after   : int
                        it - patience*eval_every, where it is the iteration in which patience hits 0,
                        or -1 if we hit max_steps before converging. 

    """
       
    ### YOUR CODE HERE ###
    num_users, num_restaurants = M.shape 
    Q, s,P = svds(M, k)
    print(P.shape)
    print(Q.shape)
    #Q = Q/num_users
    #P= P/num_restaurants
    b_q = np.zeros(num_users)
    b_p = np.zeros(num_restaurants)

    train_losses = []
    for steps in range(max_steps):
        #np.random.shuffle(samples)
        prediction1 = np.dot(Q,P)
        e = (M - prediction1)
        print((reg_lambda*P).shape)
        Q +=learning_rate*2*(-np.dot(e,P.T) + (reg_lambda*Q))   
        P += learning_rate*2*(-np.dot(Q.T,e)+(reg_lambda*P))
        predicted = np.dot(Q,P)
        error = np.sum(np.square(np.subtract(M, predicted))) + reg_lambda * np.linalg.norm(P) + reg_lambda * np.linalg.norm(Q)
        mse = np.sqrt(error)
        train_losses.append((steps, mse))
        print("Iteration: %d ; error = %.4f" % (steps, mse))
        converged_after = steps                
    best_Q = Q
    best_P = P
    validation_losses = []
    return best_Q, best_P, validation_losses, train_losses, converged_after
            


#### Train the latent factor model with alternating optimization.

a) Learn the optimal $P$ and $Q$ using standard gradient descent. That is, during each iteration you have to use all of the training examples and update $Q$ and $P$ for all users and songs at once. Try the algorithm with $k=30$, $\lambda=1$, and learning rate of 0.1. Initialize $Q$ and $P$ with SVD.  

In [53]:
Q_g_sweep, P_g_sweep, val_l_g_sweep, tr_l_g_sweep, conv_g_sweep =  latent_factor_gradient_descent(M_shifted, nonzero_indices, 
                                                                                                   k=30, val_idx=val_idx,
                                                                                                   val_values=val_values_shifted, 
                                                                                                   reg_lambda=1, learning_rate=1e-1,
                                                                                                   init='svd', batch_size=-1,
                                                                                                   max_steps=10000, log_every=20, 
                                                                                                   eval_every=20)

(30, 3531)
(11275, 30)
(30, 3531)
Iteration: 0 ; error = 1142.7737
(30, 3531)
Iteration: 1 ; error = 6294126537.8894
(30, 3531)
Iteration: 2 ; error = 1386197863961803928718641535171203956736.0000
(30, 3531)


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:103: RuntimeWarning: overflow encountered in square


Iteration: 3 ; error = inf
(30, 3531)


KeyboardInterrupt: 

#### Plot the validation and training losses over (training) time

In [ ]:

### YOUR PLOTTING CODE HERE ###


b) Learn the optimal $P$ and $Q$ using the original stochastic gradient descent (mini-batches of size 1). That is, during each iteration you sample a single random training example $r_{xi}$ and update only the respective affected parameters $\mathbf{p_x}$ and $\mathbf{q}_i$. Set the learning rate to 0.01 and keep the other parameters as in a).

In [ ]:
Q_g_st, P_g_st, val_l_g_st, tr_l_g_st, conv_g_st = latent_factor_gradient_descent(M_shifted, nonzero_indices, 
                                                                                   k=30, val_idx=val_idx,
                                                                                   val_values=val_values_shifted, 
                                                                                   reg_lambda=1, learning_rate=1e-2,
                                                                                   init='svd', batch_size=1,
                                                                                   max_steps=20000, log_every=500, 
                                                                                   eval_every=50)

#### Plot the validation and training losses over (training) time

In [ ]:

### YOUR PLOTTING CODE HERE ###


c) (**Optional**) Learn the optimal $P$ and $Q$ similarly to b) this time using larger mini-batches of size $S$, e.g. 32.

In [ ]:
Q_g_mb, P_g_mb, val_l_g_mb, tr_l_g_mb, conv_g_mb = latent_factor_gradient_descent(M_shifted, nonzero_indices, 
                                                                                   k=30, val_idx=val_idx,
                                                                                   val_values=val_values_shifted, 
                                                                                   reg_lambda=1, learning_rate=1e-1,
                                                                                   init='svd', batch_size=32,
                                                                                   max_steps=10000, log_every=100, 
                                                                                   eval_every=50)

#### Plot the validation and training losses over (training) time

In [ ]:

### YOUR PLOTTING CODE HERE ###


### 4.5 Hyperparameter search

Machine learning models are often heavily dependent on the hyperparameter settings, e.g. the learning rate. Here, we will try a simple random search to find good values of the latent factor dimension $k$, the batch size, learning rate, and regularization.  

### Tasks:

Perform a hyperparameter search to find good values for the batch size, lambda, learning rate, and latent dimension. 

* For the batch size, evaluate all values in [1, 32, 512, -1] (-1 corresponds to full-sweep gradient descent).
* For $\lambda$, randomly sample three values in the interval [0, 1).
* For the learning rate, evaluate all values in [1, 0.1, 0.01].
* For the latent dimension, uniformly sample three values in the interval [5,30].

Perform an exhaustive search among all combinations of these values;

**Hint**: This may take a while to compute. **You don't have to wait for all the models to train** -- simply use "dummy" code instead of actual model training (or let it train, e.g., for only one iteration) if you don't want to wait. Note that the signature of this dummy code has to match the function 'latent_factor_gradient_descent' so that we could simply plug in the actual function.



In [ ]:
def parameter_search(M_train, val_idx, val_values):
    """
    Hyperparameter search using random search.
    
    Parameters
    ----------
    
    M_train     : sp.spmatrix, shape [N, D]
                  Input sparse matrix where the user means have not
                  been subtracted yet. 
                  
    val_idx     : tuple, shape [2, n_validation]
                  The indices used for validation, where n_validation
                  is the size of the validation set.
                  
    val_values  : np.array, shape [n_validation, ]
                  Validation set values, where n_validation is the
                  size of the validation set.

    Returns
    -------
    best_conf   : tuple, (batch_size, lambda, learning_rate, latent_dimension)
                  The best-performing hyperparameters.
                  

    """
    
    ### YOUR CODE HERE ###
        
    print("Best configuration is {}").format(best_conf)
    return best_conf
    

In [ ]:
best_configuration = parameter_search(M_train, val_idx, val_values)

#### Output the best hyperparameter optimization

In [ ]:

### YOUR CODE HERE ###


## 5. Comparison of gradient descent and alternating optimization

After training the latent factor model with both alternating optimization and gradient descent, we now compare their results on the training, validation, and test set.

### Tasks

* Compare the root mean square errors (RMSE) for the training, validation, and test sets different settings of $k$ for both alternating optimization and gradient descent. What do you observe?
* Compare the test RMSE for the alternating optimization model and the gradient descent model. Which performs better?
* Plot the predicted ratings

**Hint**: The output values and plots below are the ones we got when testing this sheet. Yours may be different, but if your validation or test RMSE values are larger than 1.5 or 2, it is likely that you have a bug in your implementation.

In [ ]:

### YOUR CODE HERE ###


In [ ]:

### YOUR CODE HERE ###


#### Plots: Prediction vs. ground truth ratings

In [ ]:

### YOUR PLOTTING CODE HERE ###


In [ ]:

### YOUR PLOTTING CODE HERE ###


In [ ]:

### YOUR PLOTTING CODE HERE ###


In [ ]:

### YOUR PLOTTING CODE HERE ###
